In [1]:
import numpy as np

In [2]:
import os

In [3]:
dataset_filenames = os.listdir("dataset/")
categories = np.load("dataset/categories.npz")['categories']
X_train_set = []
Y_train_set = []
X_test_set = []
Y_test_set = []
for each in dataset_filenames:
    if 'test' in each:
        test_data = np.load("dataset/"+each)
        X_test_set.append(test_data['X_test'])
        Y_test_set.append(test_data['Y_test'])
    elif 'train' in each:
        train_data = np.load("dataset/"+each)
        X_train_set.append(train_data['X_train'])
        Y_train_set.append(train_data['Y_train'])
X_train = np.concatenate(X_train_set)
Y_train = np.concatenate(Y_train_set)
X_test = np.concatenate(X_test_set)
Y_test = np.concatenate(Y_test_set)

In [4]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(100000, 784) (100000, 10)
(20000, 784) (20000, 10)


In [5]:
import plaidml.keras
import os
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [32]:
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

In [14]:
K

<module 'plaidml.keras.backend' from 'C:\\Users\\rhemo\\Anaconda3\\lib\\site-packages\\plaidml\\keras\\backend.py'>

In [15]:
class Classifier:
    
    def __init__(self, model=None):
        self.model = model
        
    def get_model(self):
        return self.model
    
    def create_model():
        model = Sequential()
        model.add(Dropout(0.2, input_shape=(28,28,1)))
        model.add(Conv2D(32,kernel_size=7,padding='same',activation='relu'))
        model.add(MaxPool2D())
        model.add(Conv2D(64,kernel_size=3,padding='same',activation='relu'))
        model.add(MaxPool2D())
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(10, activation='softmax'))
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        return model
    
    def fit_model(self, X_train, Y_train, X_test, Y_test, epochs=5, batch_size=256):
        print("Starting model training")
        print("Loss values before training with test data:", self.model.evaluate(X_test, Y_test))
        self.model.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=1)
        print("Loss values after training with test data:", self.model.evaluate(X_test, Y_test))
        

In [16]:
X_train.shape

(100000, 28, 28, 1)

In [17]:
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

In [18]:
print(X_train.shape, X_test.shape)

(100000, 28, 28, 1) (20000, 28, 28, 1)


In [19]:
classifer = Classifier(Classifier.create_model())


INFO:plaidml:Opening device "opencl_amd_gfx902.0"


In [25]:
classifer.fit_model(X_train, Y_train, X_test, Y_test, batch_size=512, epochs=60)

Starting model training
20000/20000 [==============================] - 7s 355us/step
Loss values before training with test data: [0.4497741224765778, 0.8882]
Train on 90000 samples, validate on 10000 samples
Epoch 1/60
90000/90000 [==============================] - 37s 408us/step - loss: 0.1224 - acc: 0.9576 - val_loss: 0.4834 - val_acc: 0.8822
Epoch 2/60
90000/90000 [==============================] - 35s 386us/step - loss: 0.1215 - acc: 0.9580 - val_loss: 0.4980 - val_acc: 0.8798
Epoch 3/60
90000/90000 [==============================] - 32s 360us/step - loss: 0.1158 - acc: 0.9599 - val_loss: 0.4858 - val_acc: 0.8819
Epoch 4/60
90000/90000 [==============================] - 32s 353us/step - loss: 0.1106 - acc: 0.9615 - val_loss: 0.5009 - val_acc: 0.8846
Epoch 5/60
90000/90000 [==============================] - 32s 360us/step - loss: 0.1093 - acc: 0.9624 - val_loss: 0.5074 - val_acc: 0.8875
Epoch 6/60
90000/90000 [==============================] - 32s 354us/step - loss: 0.1045 - acc: 0.

90000/90000 [==============================] - 33s 367us/step - loss: 0.0538 - acc: 0.9816 - val_loss: 0.6650 - val_acc: 0.8779
Epoch 59/60
90000/90000 [==============================] - 34s 375us/step - loss: 0.0514 - acc: 0.9821 - val_loss: 0.6713 - val_acc: 0.8763
Epoch 60/60
20000/20000 [==============================] - 8s 386us/step
Loss values after training with test data: [0.6468242520451546, 0.8829]


In [29]:
model = classifer.get_model()

In [30]:
model.evaluate(X_test, Y_test)

20000/20000 [==============================] - 7s 345us/step


[0.6468242520451546, 0.8829]

In [24]:
model.save('modelv1.h5')

In [31]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'backend',
 'categories.txt',
 'Classifer Model.ipynb',
 'data',
 'dataset',
 'frontend',
 'modelv1.h5',
 'README.md',
 'requirements.txt',
 'Understanding & Preparing Data.ipynb']

In [33]:
modelv1 = load_model('modelv1.h5')

In [35]:
modelv1.evaluate(X_test, Y_test)

20000/20000 [==============================] - 7s 359us/step


[0.4497741224765778, 0.8882]